In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras import layers, callbacks

from keras.utils import to_categorical

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

from tensorflow.python.keras.utils.vis_utils import plot_model
import pydot

from scipy.stats import norm
from scipy import stats
import os

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
import dataframe_image as dfi
import pickle
import math

from keras.utils.layer_utils import count_params


In [ ]:
channel_widths = [3, 5, 7, 9, 15, 21] 
sfs = [4,6, 8, 12, 16, 32]
time_data_amounts = [8, 12, 16, 20]
nr_classes = 3
using_weights = False

### Gotta load in some data

In [ ]:
datapath = '../../../All generated data/'
labelpath = '../../../All generated labels/'
data_list = os.listdir(datapath)
#print(data_list)

#all_data = [] #if we want to have data and labels in one list
all_datapoints = []
all_labels = []

total_channels = 79
total_scans_pr_sample = 20


for csv_file in data_list:
    data_file = datapath + csv_file
    current_data_file = pd.read_csv(data_file,header=None)

    label_file = labelpath + csv_file
    label_file = label_file.replace('.csv', '_labels.csv')
    current_label_file = pd.read_csv(label_file,header=None)

    for data_iter in range(len(current_data_file.index)):
        #Pulling out the data from a row and putting it in the list
        current_data_point = np.array(current_data_file.iloc[data_iter])
        current_data_point = current_data_point.reshape(total_scans_pr_sample,total_channels)
        all_datapoints.append(current_data_point)
        
        #adding the label to the datamatrix as the last row
        label_row = np.array(current_label_file.iloc[data_iter])
        label_row = label_row.reshape(1,total_channels)
        all_labels.append(label_row)
        
        #all_data.append(np.vstack([current_data_point, label_row])) #if we want to have data and labels in one list


In [ ]:
print(all_labels[1].shape)
print(len(all_labels))
print(len(all_datapoints))

#### Pick out one channel for each sample
For now it takes the same channel for all samples

In [ ]:
for time_data_amount in time_data_amounts:
    for channel_width in channel_widths:
        for sf in sfs:
            name = 'multi_channels_test' + str(channel_width) #name of model - should be descriptive
            save_folder = 'sf'+ str(sf) + '_' + str(channel_width) + 'Ch' + str(time_data_amount) + '_VGG' #hyperparameter description her
            if using_weights == True:
                save_folder = save_folder + '_W'
                name = name + '_weighted'

            complete_data = []
            complete_labels = []

            lower_channel = math.floor(channel_width/2)
            upper_channel = math.ceil(channel_width/2)

            chosen_channels = list(range(upper_channel+1,79-upper_channel,channel_width))

            # check if channels are viable
            for channel in chosen_channels:
                    if (channel - lower_channel) < 0 or (channel + upper_channel - 1) > total_channels:
                        print('Bad channel choice')
                        exit()
                    if channel_width % 2 == 0:
                        print('please pick uneven channel width')
                        exit()


            for iter in range(len(all_datapoints)):
                for channel in chosen_channels:
                    complete_data.append(all_datapoints[iter][0:time_data_amount,channel-lower_channel:channel+upper_channel])
                    complete_labels.append(all_labels[iter][:,channel-lower_channel:channel+upper_channel])


            data_train, data_test, labels_train, labels_test = train_test_split(complete_data, complete_labels, train_size=0.8, random_state=112)

            data_train = np.array(data_train)
            data_test = np.array(data_test)
            labels_train = np.array(labels_train)
            labels_test = np.array(labels_test)


            #reshape to 1d features
            nr_data_train = data_train.shape[0]
            data_train = data_train.reshape(nr_data_train, time_data_amount*channel_width)
            nr_data_test = data_test.shape[0]
            data_test = data_test.reshape(nr_data_test, time_data_amount*channel_width)

            scaler = preprocessing.StandardScaler().fit(data_train)

            # scale everything using that scaler
            data_train = scaler.transform(data_train)
            data_test = scaler.transform(data_test)

            #reshaping back to 2d features
            data_train = data_train.reshape(nr_data_train, time_data_amount, channel_width)
            data_test = data_test.reshape(nr_data_test, time_data_amount, channel_width)

            labels_test = labels_test.reshape(nr_data_test,channel_width)
            labels_train = labels_train.reshape(nr_data_train,channel_width)
            labels_test = to_categorical(labels_test)
            labels_train = to_categorical(labels_train)

            class_weights = [0.75, 0.75, 0.9]
            class_weights = np.array([class_weights[i] for i in range(len(class_weights))])

            signal_size = time_data_amount

            y = layers.Input(shape=(signal_size,channel_width), dtype='float32', name='Input')
            x = layers.Reshape((1,time_data_amount,channel_width), input_shape=(time_data_amount,channel_width))(y)

            x = layers.Conv2D(sf, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            x = layers.Conv2D(sf, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            if channel_width == 3:
                x = layers.MaxPool2D(pool_size=[2,1], strides=[2,1], padding='valid', data_format='channels_first')(x)
            else:
                x = layers.MaxPool2D(pool_size=2, strides=2, padding='valid', data_format='channels_first')(x)
            x = layers.Conv2D(sf*2, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            x = layers.Conv2D(sf*2, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            if channel_width < 8:
                x = layers.MaxPool2D(pool_size=[2,1], strides=[2,1], padding='valid', data_format='channels_first')(x)
            else:
                x = layers.MaxPool2D(pool_size=[2, 2], strides=[2,2], padding='valid', data_format='channels_first')(x)        
            x = layers.Conv2D(sf*4, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            x = layers.Conv2D(sf*4, 3, padding='same', activation='relu', use_bias=True,data_format='channels_first')(x)
            x = layers.Flatten()(x)


            class_layer = [{}]*channel_width
            output_layer = [{}]*channel_width
            for iter in range(channel_width):
                class_layer[iter] = layers.Dropout(rate=0.5)(x)
                class_layer[iter] = layers.Dense(sf,activation='relu')(class_layer[iter])
                class_layer[iter] = layers.Dropout(rate=0.2)(class_layer[iter])
                output_layer[iter] = layers.Dense(nr_classes, activation='softmax', name=('out'+str(iter)))(class_layer[iter])



            model = Model(inputs=[y], outputs=[out_layer for out_layer in output_layer])


            isExist = os.path.exists(save_folder)
            if not isExist:
                os.makedirs(save_folder)
                print('Created "' + save_folder + '" directory')
            else:
                print('"'+ save_folder + '" directory already existed - skipping')
                continue


            nr_params = count_params(model.trainable_weights)

            cre = open(save_folder + '/' + 'Params_' + str(nr_params), 'x')


            # ------------- model compilation --------------
            ourAdam = Adam()
            optimizer = tf.keras.optimizers.RMSprop()

            if using_weights == True:
                    loss_func = weighted_mean_squared_error(class_weights)
            else:
                    loss_func = 'categorical_crossentropy'

            metric_T = 'accuracy'
            loss_dict = {}
            metric_dict = {}
            for iter in range(channel_width):
                    loss_dict['out'+str(iter)] = loss_func
                    metric_dict['out'+str(iter)] = metric_T



            model.compile(optimizer=ourAdam, loss = loss_dict,
                          metrics=metric_T) 


            if nr_params > 1000000:
                ting = 1
            elif nr_params > 500000:
                ting = 2
            elif nr_params > 250000:
                ting = 4    
            elif nr_params > 125000:
                ting = 8
            elif nr_params < 50000:
                ting = 32
            else:
                ting = 16


            BATCH_SIZE = 128*ting
            EPOCH = 300 + 75 * ting

            # Set the model training parameters
            # Stop model training when the training loss is not dropped
            callbacks_list = [callbacks.EarlyStopping(
                                    monitor='val_loss', 
                                    patience=math.floor(15 + ting * 1.3), 
                                    verbose=0, 
                                    mode='auto',
                                    restore_best_weights=True,
                                )
                                        ]

            # ------------- Starting model Training --------------

            hist = model.fit(data_train,[labels_train[:,iter,:] for iter in range(channel_width)],
                      batch_size = BATCH_SIZE, 
                      epochs = EPOCH, 
                      callbacks= callbacks_list,
                      verbose=0,
                      validation_split=0.25)


            # Show loss curves
            fig1 = plt.figure()
            plt.title('Training loss')
            plt.plot(hist.epoch, hist.history['loss'], label='train loss')
            plt.plot(hist.epoch, hist.history['val_loss'], label='val_loss')
            plt.legend()
            plt.savefig(save_folder + '/%s Training loss.pdf' %(name), format='pdf')
            #plt.show()
            plt.close(fig1)

            fig2 = plt.figure()
            plt.title('Training accuracy')
            plt.xlabel("Epoch #")
            plt.ylabel("Accuracy")

            lossNames = ['out0_accuracy', 'out'+str(math.floor(channel_width/3))+'_accuracy', 'out'+str(math.floor(channel_width*2/3))+'_accuracy']
            for (i, l) in enumerate(lossNames):
                # plot the loss for both the training and validation data
                title = "Loss for {}".format(l) if l != "loss" else "Total loss"
                plt.plot(hist.epoch, hist.history[l], label=l)
                plt.plot(hist.epoch, hist.history["val_" + l],
                    label="val_" + l)
                plt.legend()
            plt.savefig(save_folder + '/%s Training acc.pdf' %(name), format='pdf')
            #plt.show()
            plt.close(fig2)

            evalDict = model.evaluate(data_test,[labels_test[:,iter,:] for iter in range(channel_width)])
            totalA = 0
            for i in range(channel_width+1,channel_width+1+channel_width):
                totalA += evalDict[i]

            totalA /= channel_width
            print(totalA)

            # Saving dict of history and evaluation result
            with open(save_folder + '/' + 'histDict', 'wb') as file_pi:
                pickle.dump(hist.history, file_pi)

            with open(save_folder + '/' + 'evalDict' + str(totalA), 'wb') as file_pi:
                pickle.dump(evalDict, file_pi)

            model.save(save_folder + '/' + name + '_Model')

            #Test on test data
            true_test_labels = np.argmax(labels_test, axis=-1)
            test_predictions = model.predict(data_test)
            test_result = np.argmax(test_predictions, axis=-1).T

            #classification report
            class_names = ['Empty channel', 'Wi-Fi', 'Bluetooth']
            class_report = classification_report(true_test_labels.flatten(), test_result.flatten(),target_names=class_names)

            with open(save_folder + '/' + 'classReportString', 'wb') as file_pi:
                pickle.dump(class_report, file_pi)

            #Confusion matric plot
            plt.figure()
            ConfusionMatrixDisplay.from_predictions(true_test_labels.flatten(), test_result.flatten(),normalize='true',cmap='Greens',colorbar=False,display_labels=class_names)
            plt.title('Confusion Matrix')
            plt.savefig(save_folder +'/confusion_matrix_'+ name +'.pdf', format='pdf')
            plt.close()

            class_reportDict = classification_report(true_test_labels.flatten(), test_result.flatten(),output_dict=True, target_names=class_names)
            for key in class_reportDict:
                try:
                    class_reportDict[key]['Samples'] = class_reportDict[key].pop('support')
                except Exception as e:
                    print(e)
            #print(class_reportDict)
            class_reportDict.pop('accuracy')
            df = pd.DataFrame(class_reportDict).transpose().round(decimals=3)
            dfi.export(df, save_folder + '/' +name + "_ClassReport.png", table_conversion="matplotlib")
